In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
# Modify this as per your environment
%env HF_DATASETS_CACHE="/data/users/sgarg6/hf_cache"

env: HF_DATASETS_CACHE="/data/users/sgarg6/hf_cache"


<IPython.core.display.Javascript object>

In [4]:
!echo $HF_DATASETS_CACHE

"/data/users/sgarg6/hf_cache"


<IPython.core.display.Javascript object>

In [5]:
import torch
import os

device = "cuda" if torch.cuda.is_available() else "cpu"

<IPython.core.display.Javascript object>

# Code to translate the snli dataset

Code has been commented as it's no longer required. The dataset has been pushed to HF_hub and is loaded from there for all the experiments

In [6]:
# from datasets import load_dataset


# from transformers import T5Tokenizer, T5ForConditionalGeneration

# dataset = load_dataset('snli')
# tokenizer = T5Tokenizer.from_pretrained("t5-small")
# model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

# def translate(sentence):
#     sentence = ["translate English to French: " + sent for sent in sentence]
#     input_ids = tokenizer(
#         sentence, return_tensors="pt", truncation=True, padding=True
#     ).input_ids.to(device)
#     outputs = model.generate(input_ids)
#     return tokenizer.batch_decode(outputs, skip_special_tokens=True)

# def add_translations(example):
#     example["translated_premise"] = translate(example["premise"])
#     example["translated_hypothesis"] = translate(example["hypothesis"])
#     return example


# updated_dataset = dataset.map(add_translations, batched=True, batch_size=128)


# updated_dataset.save_to_disk(
#    os.path.join(os.environ["HF_DATASETS_CACHE"], "french-snli.hf")
# )

# dataset = datasets.load_from_disk(os.path.join(os.environ["HF_DATASETS_CACHE"], "french-snli.hf"))
# updated_dataset.push_to_hub("sugam11/french-snli")

<IPython.core.display.Javascript object>

In [7]:
from datasets import load_dataset

# dataset = datasets.load_from_disk(os.path.join(os.environ["HF_DATASETS_CACHE"], "french-snli.hf"))
dataset = load_dataset("sugam11/french-snli")

Using custom data configuration sugam11--french-snli-18806dba64b311d4


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /soe/sgarg6/course_work/244_nlp/french-nli/"/data/users/sgarg6/hf_cache"/sugam11___parquet/sugam11--french-snli-18806dba64b311d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

In [8]:
# Check if dataset contains all the relevant columns
dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis'],
        num_rows: 550152
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis'],
        num_rows: 10000
    })
})

<IPython.core.display.Javascript object>

In [1]:
id2label = {1: "NEUTRAL", 0: "ENTAILMENT", 2: "CONTRADICTION"}
label2id = {"ENTAILMENT": 0, "NEUTRAL": 1, "CONTRADICTION": 2}

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cmarkea/distilcamembert-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "cmarkea/distilcamembert-base", num_labels=3, id2label=id2label, label2id=label2id
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/236 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/732 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/811k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of the model checkpoint at cmarkea/distilcamembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at cmarkea/distilcamembert-base and are newly initialized: ['classifier.out_proj.weight', 'cl

<IPython.core.display.Javascript object>

In [10]:
model = model.to(device)

<IPython.core.display.Javascript object>

In [11]:
# Modify dataset for NLI task


def combine_data(sample):
    # Combine hypothesis and premise and use CLS token for classification
    text = (
        tokenizer.cls_token
        + " "
        + sample["translated_premise"]
        + " "
        + tokenizer.sep_token
        + " "
        + sample["translated_hypothesis"]
    )
    sample["text"] = text
    return sample


dataset = dataset.map(combine_data)

  0%|          | 0/550152 [00:00<?, ?ex/s]

  0%|          | 0/10000 [00:00<?, ?ex/s]

  0%|          | 0/10000 [00:00<?, ?ex/s]

<IPython.core.display.Javascript object>

In [12]:
# Tokenize the dataset

dataset = dataset.map(
    lambda x: tokenizer(x["text"], truncation=True, padding=True),
    batched=True,
    batch_size=128,
)

  0%|          | 0/4299 [00:00<?, ?ba/s]

  0%|          | 0/79 [00:00<?, ?ba/s]

  0%|          | 0/79 [00:00<?, ?ba/s]

<IPython.core.display.Javascript object>

In [13]:
# Discard data samples where label is -1
# This assumption is provided in the original SNLI dataset
dataset = dataset.filter(lambda sample: sample["label"] >= 0)

  0%|          | 0/551 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

<IPython.core.display.Javascript object>

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis', 'text', 'input_ids', 'attention_mask'],
        num_rows: 549367
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis', 'text', 'input_ids', 'attention_mask'],
        num_rows: 9842
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis', 'text', 'input_ids', 'attention_mask'],
        num_rows: 9824
    })
})

<IPython.core.display.Javascript object>

In [15]:
# convert dataset to tensors with only model inputs
dataset.set_format(type="pt", columns=["input_ids", "attention_mask", "label"])

<IPython.core.display.Javascript object>

In [16]:
dataset["train"]["label"].unique()

tensor([0, 1, 2])

<IPython.core.display.Javascript object>

In [38]:
import wandb

wandb.init(entity="sugam110795", project="nlp244", group="quest4")

train/epoch,▁▃▅▆█▁▃▅▆█
train/global_step,▁▃▅▆█▁▃▅▆█
train/learning_rate,█▆▅▃▁█▆▅▃▁
train/loss,█▅▄▄▄▃▂▁▁▁
train/epoch,0.12
train/global_step,250
train/learning_rate,4e-05
train/loss,0.5726


<IPython.core.display.Javascript object>

In [18]:
# Copied Code from https://github.com/kingb12/hf_libraries_demo

from evaluate import Metric, EvaluationModuleInfo
from typing import Dict, Any
from sklearn.metrics import f1_score
import evaluate


class MyMacroF1Metric(Metric):
    """
    You can define custom metrics! In this case I do this to compute Macro-F1, which averages per-class F1 scores
    """

    f1_metric_info: EvaluationModuleInfo = evaluate.load("f1")._info()

    def _info(self) -> EvaluationModuleInfo:
        # we'll just say the info is the same in this case
        return MyMacroF1Metric.f1_metric_info

    def _compute(
        self,
        predictions=None,
        references=None,
        labels=None,
        pos_label=1,
        sample_weight=None,
    ) -> Dict[str, Any]:
        # we can just call the sklearn implementation! Metrics in huggingface generally correspond with sklearn metrics
        # when applicable
        score = f1_score(
            references,
            predictions,
            labels=labels,
            pos_label=pos_label,
            average="macro",
            sample_weight=sample_weight,
        )
        return {"f1": float(score) if score.size == 1 else score}

<IPython.core.display.Javascript object>

In [19]:
import evaluate

# my_evaluation = evaluate.combine(["accuracy", "f1"])

my_evaluation = evaluate.combine(
    [
        evaluate.load("accuracy", average="weighted"),
        MyMacroF1Metric(),
    ]
)

<IPython.core.display.Javascript object>

In [20]:
from transformers import TrainingArguments
from sklearn.metrics import accuracy_score

training_args: TrainingArguments = TrainingArguments(
    output_dir="/data/users/sgarg6/hf_models/quest-4",
    do_train=True,
    do_eval=True,
    do_predict=True,
    evaluation_strategy="steps",
    eval_steps=256,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    save_steps=256,
    save_strategy="steps",
    save_total_limit=5,
    report_to=["wandb"],
    logging_steps=50,
    num_train_epochs=1,
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
    dataloader_num_workers=0,  # set to 0 when debugging and >1 when running!
)

<IPython.core.display.Javascript object>

In [21]:
from transformers import EvalPrediction
from typing import Dict, Tuple


def my_compute_metrics(eval_pred: EvalPrediction) -> Dict[str, float]:
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    predictions: Tensor = logits.argmax(axis=1)
    return my_evaluation.compute(predictions=predictions, references=labels)

<IPython.core.display.Javascript object>

In [22]:
from transformers import Trainer

trainer: Trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=None,  # let HF set this to an instance of transformers.DataCollatorWithPadding
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=my_compute_metrics,
)

<IPython.core.display.Javascript object>

In [111]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: translated_premise, text, premise, hypothesis, translated_hypothesis. If translated_premise, text, premise, hypothesis, translated_hypothesis are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
/soe/sgarg6/conda/envs/nlp_env/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 549367
  Num Epochs = 1
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 2146
  Number of trainable 

Step,Training Loss,Validation Loss,Accuracy,F1
256,0.571900,0.673840,0.717334,0.716320
512,0.678100,0.637535,0.731559,0.731528
768,0.651300,0.616976,0.745682,0.745820
1024,0.642000,0.610716,0.747815,0.748559
1280,0.631300,0.587332,0.758281,0.758691
1536,0.619600,0.577981,0.760516,0.760139
1792,0.621200,0.569904,0.764377,0.764184
2048,0.608800,0.569622,0.763666,0.763428


The following columns in the evaluation set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: translated_premise, text, premise, hypothesis, translated_hypothesis. If translated_premise, text, premise, hypothesis, translated_hypothesis are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9842
  Batch size = 256
Saving model checkpoint to /data/users/sgarg6/hf_models/quest-4/checkpoint-256
Configuration saved in /data/users/sgarg6/hf_models/quest-4/checkpoint-256/config.json
Model weights saved in /data/users/sgarg6/hf_models/quest-4/checkpoint-256/pytorch_model.bin
tokenizer config file saved in /data/users/sgarg6/hf_models/quest-4/checkpoint-256/tokenizer_config.json
Special tokens file saved in /data/users/sgarg6/hf_models/quest-4/checkpoint-256/special_tokens_map.json
The following columns in the evaluation set don't have a corres

TrainOutput(global_step=2146, training_loss=0.6380337664487755, metrics={'train_runtime': 585.4445, 'train_samples_per_second': 938.376, 'train_steps_per_second': 3.666, 'total_flos': 6145437024914634.0, 'train_loss': 0.6380337664487755, 'epoch': 1.0})

<IPython.core.display.Javascript object>

In [112]:
model = trainer.model

<IPython.core.display.Javascript object>

In [113]:
trainer.evaluate(metric_key_prefix="test", eval_dataset=dataset["test"])

The following columns in the evaluation set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: translated_premise, text, premise, hypothesis, translated_hypothesis. If translated_premise, text, premise, hypothesis, translated_hypothesis are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9824
  Batch size = 256


{'test_loss': 0.5801504850387573,
 'test_accuracy': 0.7595684039087948,
 'test_f1': 0.7595466066589128,
 'test_runtime': 4.3199,
 'test_samples_per_second': 2274.126,
 'test_steps_per_second': 9.028,
 'epoch': 1.0}

<IPython.core.display.Javascript object>

In [114]:
wandb.finish()

eval/accuracy,▄▄▄▄▁▄▅▅▄▅▆▇▇████
eval/f1,▂▂▁▂▄▆▆▇███
eval/loss,▄▄▅▅▅▅█▅▂▂▂▂▁▁▁▁▁▁▁
eval/runtime,▁▁▂▂▂▃▂▂█▇▂▃▄▂▂▂▂▂▃
eval/samples_per_second,██▇▇▇▅▇▇▁▂▆▆▅▇▇▆▇▆▆
eval/steps_per_second,██▇▇▇▅▇▇▁▂▆▆▅▇▇▆▇▆▆
test/accuracy,▁
test/f1,▁
test/loss,▁
test/runtime,▁
test/samples_per_second,▁


<IPython.core.display.Javascript object>